In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.cluster import MiniBatchKMeans
import numpy as np
import datetime
#import pickle

In [ ]:
def kmeans_preprocessing(start_day,end_day):
    #input filename
    start_day_str = str(start_day.date())
    end_day_str = str(end_day.date())
    train_data_filename = "../DataSet/Feature/"+"part_data_uiclf"+end_day_str+".csv"
    
    #output filename
    output_filename = "../DataSet/Kmeans/"+"part_data_uiclfk"+end_day_str+".csv"
    output_filename_uiclfc = "../DataSet/Kmeans/"+"part_data_uiclfc"+".csv"
    train_data_file_name = "../DataSet/Kmeans/"+"train_data_all"+".csv"
    
    
    start_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('start to get the uiclfk data of  %s ,start time:'%(train_data_filename[26:36]), start_time)
    
    #for train_data_set in pd.read_csv("../DataSet/train_data_set_all.csv", chunksize=15000): 
    train_data_set_sampe= pd.DataFrame()
    train_data_set = pd.read_csv(train_data_filename)
    print('train_data_set in KMeans',train_data_set.head(1))
    train_data_set.drop(train_data_set.columns[[0,1,2]], axis=1,inplace=True)    # drop user_id,item_id,cateory_id
    train_data_set_buy = train_data_set[train_data_set.label == 1]
    train_data_set_not_buy = train_data_set[train_data_set.label == 0]
    print('train_data_set_not_buy in KMeans',train_data_set_not_buy.head(1))
    train_data_set_temp = train_data_set_not_buy.iloc[:,1:]                            # drop label 
    print("train set shape-buy:",train_data_set_buy.shape)
    print("train set shape-not buy:",train_data_set_not_buy.shape)
    print("N\P ratio is:",train_data_set_not_buy.shape[0]/train_data_set_buy.shape[0])
    
    print("start to standard scale")
    scaler = preprocessing.StandardScaler()
    scaler.fit(train_data_set_temp.values)
    train_data_set_temp_values = scaler.transform(train_data_set_temp.values)
    print("start to k means algorthm")
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=1000, batch_size=500, reassignment_ratio=10**-4)
    mbk.fit(train_data_set_temp_values)
    train_data_set_temp['class'] = mbk.labels_.astype('int')+1
    train_data_set_buy['class'] = 0
    train_data_set_buy = train_data_set_buy.iloc[:,1:]   #需要确认下舍弃掉的是不是‘label’
    train_data_set_uiclfc = pd.concat([train_data_set_temp, train_data_set_buy])
    train_data_set_uiclfc.to_csv(output_filename_uiclfc, header=False, mode='a', index=False)  #有 bug。
    print('train_data_set_temp columns:\n',train_data_set_temp.columns) 
    print('columns len ',len(train_data_set_temp.columns))
    print('train_data_set_temp columns:\n',train_data_set_buy.columns)
    print('columns len ',len(train_data_set_buy.columns))
    print("start to sample from negative example of train data")
    j=0
    #print(train_data_set.head(5))
    for i in range(1,1001,1):
        j+=1
        if(j==100):
            print(i)
            j=0
        train_data_uic_label_0_i = train_data_set_temp[train_data_set_temp['class'] == i]
        #print(train_data_uic_label_0_i.shape)
        if(train_data_uic_label_0_i.shape[0]!=0):
            train_data_uic_label_0_i = train_data_uic_label_0_i.sample(frac = 0.067)
            train_data_set_sampe = pd.concat([train_data_set_sampe, train_data_uic_label_0_i])
    print("sample finished")        
    #train_data_set_sampe['class'] = (train_data_set_sampe['class']<0).astype('int')
    
   
    #train_data_set_buy['class'] = (train_data_set_buy['class']<1).astype('int')
    print("train set shape-buy:",train_data_set_buy.shape)
    print("train set shape-not buy:",train_data_set_sampe.shape)
    print("after sample N\P ratio is:",train_data_set_sampe.shape[0]/train_data_set_buy.shape[0])
    train_data_set_sampe = pd.concat([train_data_set_sampe,train_data_set_buy])
    train_data_set_sampe['class'] = (train_data_set_sampe['class']>0).astype('int')    #这不是将正负样例都变成正样例了吗？ 没有问题！！！
    train_data_set_sampe['class'] = (train_data_set_sampe['class']<1).astype('int')
    train_data_set_sampe.rename(columns={"class":"label"},inplace=True)
    print(train_data_set_sampe.shape)
    print(train_data_set_sampe.columns)
    train_data_set_sampe.to_csv(output_filename, index=False)
    end_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print('end time of getting the uiclfk data of  %s ,end time:'%(train_data_filename[26:36]), end_time)
    #将所有train data 放进同一个文件，为下一步训练模型做准备
    train_data_set_sampe.to_csv(train_data_file_name, header=False, mode='a', index=False)  #有 bug。

In [ ]:
#